In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

from matplotlib import pyplot as plt
from lifelines import CoxPHFitter
import numpy as np
import pandas as pd
from lifelines.datasets import load_rossi

plt.style.use('bmh')

## Residuals of a Cox Model

This tutorial is on some common use cases of the (many) residuals of the Cox model. We can use resdiuals to diagnose a model's poor fit to a dataset, and improve an existing model's fit. 

In [ ]:
df = load_rossi()

df['age_strata'] = pd.cut(df['age'], np.arange(0, 80, 5))
df = df.drop('age', axis=1)

cph = CoxPHFitter()
cph.fit(df, 'week', 'arrest', strata=['age_strata', 'wexp'])

In [ ]:
cph.print_summary()
cph.plot();

### Martingale residuals

Defined as:

$$ \delta_i - \Lambda(T_i) \\ = \delta_i - \beta_0(T_i)\exp(\beta^T x_i)$$

where $T_i$ is the total observation time of subject $i$ and $\delta_i$ denotes whether they died under observation of not (`event_observed` in _lifelines_). 

From [1]:

> Martingale residuals take a value between $[1,−\inf]$ for uncensored observations and $[0,−\inf]$ for censored observations. Martingale residuals can be used to assess the true functional form of a particular covariate (Thernau et al. (1990)). It is often useful to overlay a LOESS curve over this plot as they can be noisy in plots with lots of observations. Martingale residuals can also be used to assess outliers in the data set whereby the survivor function predicts an event either too early or too late, however, it's often better to use the deviance residual for this.

From [2]:

> This represents the discrepancy between the observed value of
a subject’s failure indicator and its expected value, integrated
over the time for which that patient was at risk

> Positive values mean that the patient died sooner than
expected (according to the model); negative values mean that
the patient lived longer than expected (or were censored)

In [ ]:
r = cph.compute_residuals(df, 'martingale')
r.head()

In [ ]:
r.plot.scatter(
    x='week', y='martingale', c=np.where(r['arrest'], '#008fd5', '#fc4f30'),
    alpha=0.75
)

In [ ]:
r = cph.compute_residuals(df, 'deviance')
r.head()

In [ ]:
r = r.join(df.drop(['week', 'arrest'], axis=1))


In [ ]:
plt.scatter(r['prio'], r['deviance'], color=np.where(r['arrest'], '#008fd5', '#fc4f30'))

[1] https://stats.stackexchange.com/questions/297740/what-is-the-difference-between-the-different-residuals-in-survival-analysis-cox

[2] http://myweb.uiowa.edu/pbreheny/7210/f15/notes/11-10.pdf